# Classify Movie Reviews using Embeddings

In [1]:
!pip install tensorflow==2.0.0.alpha0

    100% |████████████████████████████████| 79.9MB 367kB/s 
    100% |████████████████████████████████| 3.0MB 11.1MB/s 
    100% |████████████████████████████████| 61kB 24.4MB/s 
    100% |████████████████████████████████| 419kB 21.4MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [6]:
#this version of numpy is required to avoid an error related to numpy defaulting to not allowing pickle files
!pip install numpy==1.16.2

    100% |████████████████████████████████| 17.3MB 2.1MB/s 
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3


In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, preprocessing, models, layers
import numpy as np

## Import IMDB Reviews

In [0]:
imdb = datasets.imdb

#Let's work with a dictionary of 20,000 words
NUM_WORDS = 20000

In [0]:
#load IMDB dataset as lists of integers
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, )

## Get Word Index To See Words

In [4]:
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k: (v+3) for k, v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

1646592/1641221 [==============================] - 0s 0us/step


## Inspect First Review

In [5]:
print(' '.join([reverse_word_index[i] for i in train_data[0]]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be p

## Was this a positive review?

In [6]:
print(train_labels[0])

1


## Note: Reviews have variable length

In [7]:
print('Len 0: {}, Len 1: {}, Len 2: {}'.format(len(train_data[0]), len(train_data[1]), len(train_data[2])))

Len 0: 218, Len 1: 189, Len 2: 141


Variable lenght is fixed by truncating after a certain number of words. For reviews that are less than the number of words we are cutting off, we pad.

In [0]:
LEN_WORDS = 300
train_data = preprocessing.sequence.pad_sequences(train_data, maxlen=LEN_WORDS)
test_data = preprocessing.sequence.pad_sequences(test_data, maxlen=LEN_WORDS)

In [9]:
print('Len 0: {}, Len 1: {}, Len 2: {}'.format(len(train_data[0]), len(train_data[1]), len(train_data[2])))

Len 0: 300, Len 1: 300, Len 2: 300


In [13]:
print(train_data.shape)

(25000, 300)


## Sequential Model with Dense Layers

In [30]:
dense_model = models.Sequential([
    layers.Dense(300, input_shape=(300,), activation='relu'),
    layers.Dense(300, activation='relu'),
    layers.Dense(300, activation='relu'),
    layers.Dense(300, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
print(dense_model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_23 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_24 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_25 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 301       
Total params: 361,501
Trainable params: 361,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
dense_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
dense_model.fit(train_data, train_labels, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 5s 187us/sample - loss: 14.8637 - accuracy: 0.4971
Epoch 2/10
25000/25000 [==============================] - 4s 176us/sample - loss: 1.1852 - accuracy: 0.5194
Epoch 3/10
25000/25000 [==============================] - 4s 174us/sample - loss: 0.8335 - accuracy: 0.5318
Epoch 4/10
25000/25000 [==============================] - 4s 174us/sample - loss: 0.7373 - accuracy: 0.5578
Epoch 5/10
25000/25000 [==============================] - 4s 174us/sample - loss: 0.7519 - accuracy: 0.5475
Epoch 6/10
25000/25000 [==============================] - 4s 173us/sample - loss: 0.7289 - accuracy: 0.5349
Epoch 7/10
25000/25000 [==============================] - 4s 173us/sample - loss: 0.7184 - accuracy: 0.5152
Epoch 8/10
25000/25000 [==============================] - 4s 174us/sample - loss: 0.6946 - accuracy: 0.5022
Epoch 9/10
25000/25000 [==============================] - 4s 173us/sample - loss: 0.6948 - accuracy: 0.4977
Epoch 10/10
25000/25000 [==

## Introduce Embeddings

In [27]:
DIMENSION = 16

e_model = models.Sequential([
    layers.Embedding(NUM_WORDS, DIMENSION, input_length=LEN_WORDS),
    layers.GlobalAveragePooling1D(),
    layers.Dense(1, activation='sigmoid')
])
print(e_model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 320,017
Trainable params: 320,017
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
e_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
e_model.fit(train_data, train_labels, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 5s 203us/sample - loss: 0.6439 - accuracy: 0.7200
Epoch 2/10
25000/25000 [==============================] - 5s 195us/sample - loss: 0.4831 - accuracy: 0.8366
Epoch 3/10
25000/25000 [==============================] - 5s 197us/sample - loss: 0.3724 - accuracy: 0.8734
Epoch 4/10
25000/25000 [==============================] - 5s 193us/sample - loss: 0.3104 - accuracy: 0.8931
Epoch 5/10
25000/25000 [==============================] - 5s 196us/sample - loss: 0.2707 - accuracy: 0.9046
Epoch 6/10
25000/25000 [==============================] - 5s 195us/sample - loss: 0.2419 - accuracy: 0.9152
Epoch 7/10
25000/25000 [==============================] - 5s 197us/sample - loss: 0.2187 - accuracy: 0.9240
Epoch 8/10
25000/25000 [==============================] - 5s 193us/sample - loss: 0.1993 - accuracy: 0.9310
Epoch 9/10
25000/25000 [==============================] - 5s 194us/sample - loss: 0.1822 - accuracy: 0.9386
Epoch 10/10
25000/25000 [===